# GRIB2-CMA GRAPES GEPS v1.3
This notebook provides an example on how to decode GRIB2 data using the pygrib library. In this example, we will read GEPS data from the CMA Global Regional Assimilation PrEdiction System (GRAPES) and display all variables on maps.<br>
<br>
As part of the WIS2 Training you will be asked to download some data from the WIS2-broker hosted at wis2training-broker.wis2dev.io and use it in this notebook. If you want to run this notebook on your local machine, you can use the sample-data included in this notebook instead.<br>
<br>
The following references were used to create this notebook:<br>
- Sample data from <https://wis2node.wis.cma.cn/oapi/collections/discovery-metadata/items/urn:wmo:md:cn-cma:data.core.weather.prediction.forecast.medium-range.probabilistic.global>
- Python code to load GRIB data, see <https://pypi.org/project/pygrib/><br>

Execute each block of code by selecting it and pressing ``Shift+Enter`` or clicking the ``Run`` button in the toolbar above.

# Data decode
The following block imports the required modules for reading GRIB data, numerical computing, visualization, and cartographic mapping:  

The following modules are imported:  

- **pygrib**: Python interface for reading and writing GRIB (Gridded Binary) files, enabling extraction of meteorological data (e.g., loading forecast fields like temperature or wind).  
- **numpy**: Numerical Python library for efficient array operations and mathematical computations (e.g., `np.array` for handling gridded data).  
- **matplotlib**: Plotting library for creating static, interactive, or animated visualizations (e.g., `plt` for rendering maps or graphs).  
- **cartopy**: Cartographic library for geospatial data visualization, offering map projections and geographic features (e.g., `ccrs` for coordinate reference systems, `cfeature` for adding coastlines or borders).  

This setup is typically used for analyzing and visualizing meteorological or climate data stored in GRIB format, with **pygrib** handling data ingestion and the remaining modules supporting processing and mapping tasks.


## **pygrib**  
**Python library for reading GRIB files (editions 1 and 2)**, enabling access to meteorological and climate data stored in GRIB format. Requires the **ECCODES C library** as a dependency.  

pygrib is included in the `requirements.txt` file and installed of the docker image running this notebook.

### Installing pygrib outside this docker image
For Anaconda/Miniconda users, the simplest method is:  
```bash
conda install -c conda-forge pygrib
```  
This automatically resolves dependencies (e.g., ECCODES, numpy, pyproj).  

### **Verification**  
Test installation with:  
```python
import pygrib
print(pygrib.__version__)  # Should output the installed version (e.g., `2.1.6`)
```  

In [ ]:
import pygrib
import numpy as np 

#grib_file = "Z_NAFP_C_BABJ_20250818000000_P_CMA-GEPS-GLB-036.grib2"
grib_file = "/root/sample-data/Z_NAFP_C_BABJ_20250818000000_P_CMA-GEPS-GLB-024.grib2"

In [ ]:
grbs = pygrib.open(grib_file)
print("GRIB file contents:")
for grb in grbs:
    print(grb)

In [ ]:
try:
    grbs = pygrib.open(grib_file)
      
    for idx in range(1, len(list(grbs)) + 1):
        grb = grbs.message(idx)
        
        try:
            data, lats, lons = grb.data()
            has_data = True
        except Exception as e:
            has_data = False
            print(f"\nMessage {idx} Warning: Failed to extract data - {str(e)}")
        
        print(f"\nMessage {idx} Details:")
        print(f"Parameter Name: {grb.parameterName}")
        print(f"Units: {grb.units}")
        print(f"Type of Level: {grb.typeOfLevel}")
        print(f"Level: {grb.level}")
        
        if has_data:
            print(f"Data Shape: {data.shape}")
            print(f"Latitude Range: {np.nanmin(lats):.2f} ~ {np.nanmax(lats):.2f}")
            print(f"Longitude Range: {np.nanmin(lons):.2f} ~ {np.nanmax(lons):.2f}")
            print(f"Data Min: {np.nanmin(data):.2f}")
            print(f"Data Max: {np.nanmax(data):.2f}")
        else:
            print("No grid data available for this message")

    grbs.close()

except FileNotFoundError:
    print(f"Error: The file '{grib_file}' was not found.")
except pygrib.PyGribError as e:
    print(f"PyGrib Error: Failed to read GRIB file - {str(e)}")
except Exception as e:
    print(f"An unexpected error occurred: {str(e)}")

# Visualization

In [ ]:
import pygrib
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from matplotlib.colorbar import ColorbarBase
from matplotlib.patches import Rectangle

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [ ]:
grbs = pygrib.open(grib_file)

def plot_grib_variable(grb, title, cmap="viridis"):
    """
    Visualize GRIB variable (display only, no saving)
    :param grb: pygrib object
    :param title: plot title
    :param cmap: colormap
    """
    # Extract data, latitude, longitude
    data, lats, lons = grb.data()
    data_min, data_max = np.nanmin(data), np.nanmax(data)
    
    # Create figure and axes
    fig = plt.figure(figsize=(12, 8))
    ax = plt.axes(projection=ccrs.PlateCarree())
    
    # Add map features
    ax.add_feature(cfeature.COASTLINE, linewidth=0.5)
    ax.add_feature(cfeature.BORDERS, linestyle=":", linewidth=0.5)
    ax.add_feature(cfeature.LAND, edgecolor="black", facecolor="lightgray")
    ax.add_feature(cfeature.LAKES, alpha=0.5)
    
    # Ensure global display
    ax.set_global()
    
    # Plot data
    mesh = ax.pcolormesh(
        lons, lats, data,
        cmap=cmap,
        vmin=data_min, vmax=data_max,
        transform=ccrs.PlateCarree()
    )
    
    # Adjust colorbar position and length to match map width
    cbar_ax = fig.add_axes([0.1, 0.05, 0.8, 0.02])  # [left, bottom, width, height]
    cbar = fig.colorbar(
        mesh,
        cax=cbar_ax,
        orientation="horizontal",
        extend="both"
    )
    cbar.set_label(f"{grb.parameterName} ({grb.units})", fontsize=10)
    
    # Display min/max values below colorbar
    cbar_ax.text(
        0.5, 2,  # Adjust text position
        f"Min: {data_min:.2f}  Max: {data_max:.2f}",
        transform=cbar_ax.transAxes,
        ha="center", va="center",
        fontsize=9,
        bbox=dict(facecolor="white", alpha=0.8, edgecolor="gray", boxstyle="round")
    )
    
    # Set title
    ax.set_title(title, fontsize=14, pad=20)
    
    # Display plot (no saving)
    plt.tight_layout()
    plt.show()

# Iterate through all variables and visualize
for i, grb in enumerate(grbs, 1):
    print(f"Processing variable {i}: {grb.parameterName}")
    
    try:
        data, lats, lons = grb.data()
        if np.all(np.isnan(data)) or "Missing" in str(data):
            print(f"Variable {i} has invalid data, skipping")
            continue
        
        # Select colormap based on variable type
        if "precipitation" in grb.parameterName.lower():
            cmap = "Blues"
        elif "temperature" in grb.parameterName.lower():
            cmap = "coolwarm"
        elif "wind" in grb.parameterName.lower() or "gust" in grb.parameterName.lower():
            cmap = "YlOrRd"
        else:
            cmap = "viridis"
        
        # Generate title
        title = f"{grb.parameterName}\n{grb.level} hPa | {grb.validDate}"
        
        # Call plotting function (no saving)
        plot_grib_variable(grb, title, cmap)
    
    except Exception as e:
        print(f"Failed to process variable {i}: {e}")

# Close GRIB file
grbs.close()

You can also visualise only one parameter.

In [ ]:
grbs = pygrib.open(grib_file)
# Filter precipitation-related variables
precip_vars = []
for grb in grbs:
    if "Total precipitation" in grb.parameterName:
        precip_vars.append(grb)
        if len(precip_vars) >= 6:
            break

if len(precip_vars) < 6:
    raise ValueError(f"Only found {len(precip_vars)} precipitation variables, need at least 6!")

# Create figure and subplots
fig, axes = plt.subplots(
    nrows=3, ncols=2,
    figsize=(14, 18),
    subplot_kw={"projection": ccrs.PlateCarree()}
)
fig.subplots_adjust(hspace=0.4, wspace=0.3)

cmap = "Blues"

for i, (grb, ax) in enumerate(zip(precip_vars, axes.flat), 1):
    # Extract data
    data, lats, lons = grb.data()
    data_min, data_max = np.nanmin(data), np.nanmax(data)
    
    # Ensure global display
    ax.set_global()
    
    # Add map features
    ax.add_feature(cfeature.COASTLINE, linewidth=0.5)
    ax.add_feature(cfeature.BORDERS, linestyle=":", linewidth=0.5)
    ax.add_feature(cfeature.LAND, edgecolor="black", facecolor="lightgray")
    
    # Plot data
    mesh = ax.pcolormesh(
        lons, lats, data,
        cmap=cmap,
        vmin=data_min, vmax=data_max,
        transform=ccrs.PlateCarree()
    )
    
    # Create colorbar matching subplot width
    pos = ax.get_position()
    cax = fig.add_axes([pos.x0, pos.y0 - 0.08, pos.width, 0.01])  # Adjust colorbar position and height
    
    # Create colorbar
    norm = plt.Normalize(vmin=data_min, vmax=data_max)
    ColorbarBase(cax, cmap=cmap, norm=norm, orientation='horizontal', extend='both')
    cax.set_title(f"{grb.parameterName} ({grb.units})", fontsize=8)
    
    # Display min/max values
    ax.text(
        0.5, -0.25,
        f"Min: {data_min:.2f}\nMax: {data_max:.2f}",
        transform=ax.transAxes,
        ha="center", va="center",
        fontsize=8,
        bbox=dict(facecolor="white", alpha=0.8, edgecolor="gray", boxstyle="round")
    )
    
    # Set title
    ax.set_title(f"Variable {i}: {grb.parameterName}\n{grb.validDate}", fontsize=10)

# Close unused subplots
for j in range(len(precip_vars), 6):
    fig.delaxes(axes.flat[j])

# Display plot
plt.suptitle("Total Precipitation (12-36 hrs Forecast)", fontsize=16, y=0.9)
plt.show()

grbs.close()